In [17]:
import pandas as pd
import os
import json
from convlab.base_models.t5.mdst.evaluate_qa import eval_slot_pairs_prediction

### Main Exp Table

In [8]:
aug_types = [(0, 0.0), (2, 2.0), (5, 2.0)]#, (1, 0.1)]
index_names = ['Single', 'Single+Concat', 'Single+ConcatRel']#, '90%Single+10%Multi']
column_names = ['RSA', 'CDTA']
datasets = ['sgd/group0', 'multiwoz21']
models = ['T5DST', 'MinTL', 'SDPDST']

In [13]:
main_exp_table = pd.DataFrame(index=pd.MultiIndex.from_product([models, index_names, [0, 4]]), 
                              columns=pd.MultiIndex.from_product([datasets, column_names, [f'domain_{i}' for i in range(1,5)]]))
main_exp_table

sgd/group0                                      \
                                 RSA                                CDTA   
                            domain_1 domain_2 domain_3 domain_4 domain_1   
T5DST  Single           0        NaN      NaN      NaN      NaN      NaN   
                        4        NaN      NaN      NaN      NaN      NaN   
       Single+Concat    0        NaN      NaN      NaN      NaN      NaN   
                        4        NaN      NaN      NaN      NaN      NaN   
       Single+ConcatRel 0        NaN      NaN      NaN      NaN      NaN   
                        4        NaN      NaN      NaN      NaN      NaN   
MinTL  Single           0        NaN      NaN      NaN      NaN      NaN   
                        4        NaN      NaN      NaN      NaN      NaN   
       Single+Concat    0        NaN      NaN      NaN      NaN      NaN   
                        4        NaN      NaN      NaN      NaN      NaN   
       Single+ConcatRel 0        NaN      NaN      NaN      NaN      NaN   
                        4        NaN      NaN      NaN      NaN      NaN   
SDPDST Single           0        NaN      NaN      NaN      NaN      NaN   
                        4        NaN      NaN      NaN      NaN      NaN   
       Single+Concat    0        NaN      NaN      NaN      NaN      NaN   
                        4        NaN      NaN      NaN      NaN      NaN   
       Single+ConcatRel 0        NaN      NaN      NaN      NaN      NaN   
                        4        NaN      NaN      NaN      NaN      NaN   

                                                     multiwoz21           \
                                                            RSA            
                          domain_2 domain_3 domain_4   domain_1 domain_2   
T5DST  Single           0      NaN      NaN      NaN        NaN      NaN   
                        4      NaN      NaN      NaN        NaN      NaN   
       Single+Concat    0      NaN      NaN      NaN        NaN      NaN   
                        4      NaN      NaN      NaN        NaN      NaN   
       Single+ConcatRel 0      NaN      NaN      NaN        NaN      NaN   
                        4      NaN      NaN      NaN        NaN      NaN   
MinTL  Single           0      NaN      NaN      NaN        NaN      NaN   
                        4      NaN      NaN      NaN        NaN      NaN   
       Single+Concat    0      NaN      NaN      NaN        NaN      NaN   
                        4      NaN      NaN      NaN        NaN      NaN   
       Single+ConcatRel 0      NaN      NaN      NaN        NaN      NaN   
                        4      NaN      NaN      NaN        NaN      NaN   
SDPDST Single           0      NaN      NaN      NaN        NaN      NaN   
                        4      NaN      NaN      NaN        NaN      NaN   
       Single+Concat    0      NaN      NaN      NaN        NaN      NaN   
                        4      NaN      NaN      NaN        NaN      NaN   
       Single+ConcatRel 0      NaN      NaN      NaN        NaN      NaN   
                        4      NaN      NaN      NaN        NaN      NaN   

                                                                        \
                                                CDTA                     
                          domain_3 domain_4 domain_1 domain_2 domain_3   
T5DST  Single           0      NaN      NaN      NaN      NaN      NaN   
                        4      NaN      NaN      NaN      NaN      NaN   
       Single+Concat    0      NaN      NaN      NaN      NaN      NaN   
                        4      NaN      NaN      NaN      NaN      NaN   
       Single+ConcatRel 0      NaN      NaN      NaN      NaN      NaN   
                        4      NaN      NaN      NaN      NaN      NaN   
MinTL  Single           0      NaN      NaN      NaN      NaN      NaN   
                        4      NaN      NaN      NaN      NaN      NaN   
       Single+Conc

In [14]:
table = main_exp_table
for i, (aug_type, aug_times) in enumerate(aug_types):
    index_name = index_names[i]
    for j, model_name in zip([0,1,3], models):
        for k in [j, j+4]:
            c = 4 if k == 1 else 100
            for dataset in datasets:
                res_dir = f'output0615/{dataset}/qadst_f1_th>0.1/t5-large_aug{aug_type}_x{aug_times}_model{k}_context{c}'
                
                res_file = f'{res_dir}/test_multi_domain_result_ori_cdta.md'
                if not os.path.exists(res_file):
                    continue
                df = pd.read_table(res_file, sep='|', index_col=1).dropna(axis=1, how='all').iloc[1:]
                df = df.apply(lambda x: x.str.strip())
                df = df.rename(columns=str.strip).rename(index=str.strip)

                for dom_cnt in range(1, 5):
                    table.loc[(model_name, index_name, k-j), (dataset, 'RSA', f'domain_{dom_cnt}')] = df.loc[f'domain_{dom_cnt}', 'RSA']
                    table.loc[(model_name, index_name, k-j), (dataset, 'CDTA', f'domain_{dom_cnt}')] = df.loc[f'domain_{dom_cnt}', 'CDTA']
                # table.loc[(model_name, index_name, k-j), (dataset, 'CDTA', f'domain_{1}')] = df.loc['all', 'RSA']
                table.loc[(model_name, index_name, k-j), (dataset, 'CDTA', f'domain_{1}')] = df.loc['all', 'CDTA']

main_exp_table = main_exp_table.applymap(lambda x: round(pd.to_numeric(x, errors='coerce') * 100, 1))

/tmp/ipykernel_33250/1508128031.py:18: PerformanceWarning: indexing past lexsort depth may impact performance.
  table.loc[(model_name, index_name, k-j), (dataset, 'RSA', f'domain_{dom_cnt}')] = df.loc[f'domain_{dom_cnt}', 'RSA']
/tmp/ipykernel_33250/1508128031.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  table.loc[(model_name, index_name, k-j), (dataset, 'CDTA', f'domain_{dom_cnt}')] = df.loc[f'domain_{dom_cnt}', 'CDTA']
/tmp/ipykernel_33250/1508128031.py:21: PerformanceWarning: indexing past lexsort depth may impact performance.
  table.loc[(model_name, index_name, k-j), (dataset, 'CDTA', f'domain_{1}')] = df.loc['all', 'CDTA']
/tmp/ipykernel_33250/1508128031.py:18: PerformanceWarning: indexing past lexsort depth may impact performance.
  table.loc[(model_name, index_name, k-j), (dataset, 'RSA', f'domain_{dom_cnt}')] = df.loc[f'domain_{dom_cnt}', 'RSA']
/tmp/ipykernel_33250/1508128031.py:19: PerformanceWarning: indexing past lexsort depth may impac

In [15]:
main_exp_table = main_exp_table.drop(columns=[('multiwoz21', 'RSA', 'domain_4'),
                                              ('multiwoz21', 'CDTA', 'domain_2'), 
                                              ('multiwoz21', 'CDTA', 'domain_3'), ('multiwoz21', 'CDTA', 'domain_4'),
                                              ('sgd/group0', 'CDTA', 'domain_2'), 
                                              ('sgd/group0', 'CDTA', 'domain_3'), ('sgd/group0', 'CDTA', 'domain_4'),
                                              ])
# for i, (aug_type, aug_times) in enumerate(aug_types):
#     index_name = index_names[i]
#     main_exp_table = main_exp_table.drop(index=('MinTL', index_name, 4))

In [16]:
main_exp_table #0620 ori cdta

sgd/group0                                      \
                                 RSA                                CDTA   
                            domain_1 domain_2 domain_3 domain_4 domain_1   
T5DST  Single           0       94.1     44.9     23.5     13.2      2.7   
                        4       93.8     71.8     59.5     48.4     11.8   
       Single+Concat    0       94.0     78.1     53.5     32.9      1.6   
                        4       93.9     76.9     74.7     50.9      1.8   
       Single+ConcatRel 0       94.0     81.8     54.3     33.1     25.8   
                        4       93.1     80.2     77.3     65.5     21.1   
MinTL  Single           0       92.3     47.4     29.8     12.9      2.1   
                        4       93.5     64.3     56.2     37.0      0.8   
       Single+Concat    0       92.8     70.0     67.8     54.3      1.1   
                        4       93.4     74.4     72.2     58.3      0.5   
       Single+ConcatRel 0       91.7     70.2     64.0     54.5     18.2   
                        4       92.9     78.0     73.4     62.9     21.8   
SDPDST Single           0       93.6     59.0     44.7     47.7      3.8   
                        4       94.7     74.7     66.2     54.6     13.2   
       Single+Concat    0       93.3     81.9     79.5     66.6      0.8   
                        4       95.0     78.3     77.5     58.7      1.7   
       Single+ConcatRel 0       92.6     83.4     80.1     74.6     18.1   
                        4       94.8     81.9     78.4     66.4     19.7   

                          multiwoz21                             
                                 RSA                       CDTA  
                            domain_1 domain_2 domain_3 domain_1  
T5DST  Single           0       85.9     44.4     22.1      1.6  
                        4       85.1     65.2     54.1      6.6  
       Single+Concat    0       85.3     83.9     44.9     11.2  
                        4       85.3     83.6     68.4     11.1  
       Single+ConcatRel 0       85.1     82.4     50.7     20.9  
                        4       84.7     80.5     70.0     27.6  
MinTL  Single           0       83.0     52.7     37.2      1.8  
                        4       83.9     59.3     44.4      1.9  
       Single+Concat    0       84.2     81.5     64.0      5.8  
                        4       84.4     82.0     65.1      7.3  
       Single+ConcatRel 0       83.5     80.0     66.3     27.5  
                        4       83.6     80.4     66.3     29.1  
SDPDST Single           0       86.2     58.0     39.8      1.4  
                        4       86.2     66.9     54.0      8.5  
       Single+Concat    0       87.2     84.9     68.5      9.1  
                        4       86.6     84.3     68.1      8.3  
       Single+ConcatRel 0       86.9     83.8     72.4     26.8  
                        4       86.2     82.6     70.6     27.7

In [38]:
main_exp_table #0617 fix empty value bug

sgd/group0                                      \
                                 RSA                                CDTA   
                            domain_1 domain_2 domain_3 domain_4 domain_1   
T5DST  Single           0       94.1     44.9     23.5     13.2      4.9   
                        4       93.8     71.8     59.5     48.4     15.8   
       Single+Concat    0       94.0     78.1     53.5     32.9      5.5   
                        4       93.9     76.9     74.7     50.9      8.8   
       Single+ConcatRel 0       94.0     81.8     54.3     33.1     29.0   
                        4       92.9     81.3     77.0     66.7     31.0   
MinTL  Single           0       92.3     47.4     29.8     12.9      4.5   
                        4       93.5     64.3     56.2     37.0      5.6   
       Single+Concat    0       92.8     70.0     67.8     54.3      7.1   
                        4       93.4     74.4     72.2     58.3      7.6   
       Single+ConcatRel 0       91.7     70.2     64.0     54.5     22.9   
                        4       92.9     78.0     73.4     62.9     26.7   
SDPDST Single           0       93.6     59.0     44.7     47.7      6.7   
                        4       94.7     74.7     66.2     54.6     17.4   
       Single+Concat    0       93.3     81.9     79.5     66.6      7.8   
                        4       95.0     78.3     77.5     58.7      9.9   
       Single+ConcatRel 0       92.6     83.4     80.1     74.6     24.1   
                        4       94.6     82.5     78.8     65.8     25.8   

                          multiwoz21                             
                                 RSA                       CDTA  
                            domain_1 domain_2 domain_3 domain_1  
T5DST  Single           0       85.9     44.4     22.1      4.1  
                        4       85.1     65.2     54.1     18.4  
       Single+Concat    0       85.3     83.9     44.9     32.4  
                        4       85.3     83.6     68.4     37.3  
       Single+ConcatRel 0       85.1     82.4     50.7     35.4  
                        4       84.8     81.8     70.4     45.5  
MinTL  Single           0       83.0     52.7     37.2      3.2  
                        4       83.9     59.3     44.4      4.7  
       Single+Concat    0       84.2     81.5     64.0     30.9  
                        4       84.4     82.0     65.1     35.6  
       Single+ConcatRel 0       83.5     80.0     66.3     42.0  
                        4       83.6     80.4     66.3     46.4  
SDPDST Single           0       86.2     58.0     39.8      7.6  
                        4       86.2     66.9     54.0     23.0  
       Single+Concat    0       87.2     84.9     68.5     34.7  
                        4       86.6     84.3     68.1     36.2  
       Single+ConcatRel 0       86.9     83.8     72.4     42.5  
                        4       85.9     82.3     70.3     44.3

In [73]:
main_exp_table #0615

sgd/group0                                      \
                                 RSA                                CDTA   
                            domain_1 domain_2 domain_3 domain_4 domain_1   
MinTL  Single           0       93.5     49.3     30.5     26.9      4.0   
                        4       93.3     63.0     53.4     27.8      5.1   
       Single+Concat    0       93.1     69.6     66.2     53.6      7.5   
                        4       93.2     74.1     72.4     58.1      7.3   
       Single+ConcatRel 0       92.5     69.8     64.2     52.5     19.2   
                        4       92.8     78.3     73.6     61.2     26.5   
T5DST  Single           0       94.1     44.9     23.5     13.2      4.9   
                        4       93.8     71.8     59.5     48.4     15.8   
       Single+Concat    0       94.0     78.1     53.5     32.9      5.5   
                        4       93.9     76.9     74.7     50.9      8.8   
       Single+ConcatRel 0       94.0     81.8     54.3     33.1     29.0   
                        4       92.9     81.3     77.0     66.7     31.0   
SDPDST Single           0       93.6     59.0     44.7     47.7      6.7   
                        4       94.7     74.7     66.2     54.6     17.4   
       Single+Concat    0        NaN      NaN      NaN      NaN      NaN   
                        4       95.0     78.3     77.5     58.7      9.9   
       Single+ConcatRel 0        NaN      NaN      NaN      NaN      NaN   
                        4       94.3     81.9     79.3     69.5     25.4   

                          multiwoz21                                      
                                 RSA                                CDTA  
                            domain_1 domain_2 domain_3 domain_4 domain_1  
MinTL  Single           0       83.2     52.4     38.2     28.3      2.5  
                        4       84.7     58.2     42.5     38.3      3.3  
       Single+Concat    0       84.6     82.0     64.6     74.3     32.7  
                        4       84.4     82.3     66.0     83.0     36.3  
       Single+ConcatRel 0       83.5     80.2     66.2     63.6     43.7  
                        4       83.9     80.3     67.1     74.4     45.2  
T5DST  Single           0       85.9     44.4     22.1     15.7      4.1  
                        4       85.1     65.2     54.1     51.9     18.4  
       Single+Concat    0       85.3     83.9     44.9     38.5     32.4  
                        4       85.3     83.6     68.4     77.6     37.3  
       Single+ConcatRel 0       85.1     82.4     50.7     29.3     35.4  
                        4       84.8     81.8     70.4     81.4     45.5  
SDPDST Single           0       86.2     58.0     39.8     25.6      7.6  
                        4       86.2     66.9     54.0     61.9     23.0  
       Single+Concat    0       87.2     84.9     68.5     74.0     34.7  
                        4       86.6     84.3     68.1     83.0     36.2  
       Single+ConcatRel 0       87.3     83.5     70.7     73.6     41.7  
                        4       86.2     82.4     67.7     74.7     43.9

In [20]:
print(main_exp_table.to_latex())

\begin{tabular}{lllrrrrrrrrr}
\toprule
       &                  &   & \multicolumn{5}{l}{sgd/group0} & \multicolumn{4}{l}{multiwoz21} \\
       &                  &   & \multicolumn{4}{l}{RSA} &     CDTA & \multicolumn{3}{l}{RSA} &     CDTA \\
       &                  &   &   domain\_1 & domain\_2 & domain\_3 & domain\_4 & domain\_1 &   domain\_1 & domain\_2 & domain\_3 & domain\_1 \\
\midrule
T5DST & Single & 0 &       94.1 &     44.9 &     23.5 &     13.2 &      2.7 &       85.9 &     44.4 &     22.1 &      1.6 \\
       &                  & 4 &       93.8 &     71.8 &     59.5 &     48.4 &     11.8 &       85.1 &     65.2 &     54.1 &      6.6 \\
       & Single+Concat & 0 &       94.0 &     78.1 &     53.5 &     32.9 &      1.6 &       85.3 &     83.9 &     44.9 &     11.2 \\
       &                  & 4 &       93.9 &     76.9 &     74.7 &     50.9 &      1.8 &       85.3 &     83.6 &     68.4 &     11.1 \\
       & Single+ConcatRel & 0 &       94.0 &     81.8 &     54.3 &     

/tmp/ipykernel_50930/1754652341.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(main_exp_table.to_latex())


### Domain expand

In [21]:
aug_types = [(18, 2.0), (19, 2.0), (5, 2.0), (14, 2.0), (15, 2.0), (1, 0.1)][:3]
index_names = ['10SYN', '15SYN', 'SYN', '10TURE', '15TRUE', '19TRUE'][:3]
datasets = ['sgd/group0']
models = ['T5DST']
domain_expand_table = pd.DataFrame(index=pd.MultiIndex.from_product([[0, 4], index_names]), 
                                   columns=pd.MultiIndex.from_product([['RSA', 'CDTA'], ['sgd10', 'sgd15', 'sgd19', 'all']]))
domain_expand_table

RSA                   CDTA                 
        sgd10 sgd15 sgd19  all sgd10 sgd15 sgd19  all
0 10SYN   NaN   NaN   NaN  NaN   NaN   NaN   NaN  NaN
  15SYN   NaN   NaN   NaN  NaN   NaN   NaN   NaN  NaN
  SYN     NaN   NaN   NaN  NaN   NaN   NaN   NaN  NaN
4 10SYN   NaN   NaN   NaN  NaN   NaN   NaN   NaN  NaN
  15SYN   NaN   NaN   NaN  NaN   NaN   NaN   NaN  NaN
  SYN     NaN   NaN   NaN  NaN   NaN   NaN   NaN  NaN

In [22]:
table = domain_expand_table
for i, (aug_type, aug_times) in enumerate(aug_types):
    index_name = index_names[i]
    for j, model_name in zip([0,1,3], models):
        for k in [j, j+4]:
            c = 4 if k == 1 else 100
            for dataset in datasets:
                res_dir = f'output0615/{dataset}/qadst_f1_th>0.1/t5-large_aug{aug_type}_x{aug_times}_model{k}_context{c}'
                
                res_file = f'{res_dir}/test_multi_domain_result_ori_cdta.md'
                if not os.path.exists(res_file):
                    continue
                df = pd.read_table(res_file, sep='|', index_col=1).dropna(axis=1, how='all').iloc[1:]
                df = df.apply(lambda x: x.str.strip())
                df = df.rename(columns=str.strip).rename(index=str.strip)

                for group in ['sgd10', 'sgd15', 'sgd19', 'all']:
                    table.loc[(k-j, index_name), ('RSA', group)] = df.loc[group, 'RSA']
                    table.loc[(k-j, index_name), ('CDTA', group)] = df.loc[group, 'CDTA']

domain_expand_table = domain_expand_table.applymap(lambda x: round(pd.to_numeric(x, errors='coerce') * 100, 1))

In [23]:
domain_expand_table # 0620

RSA                    CDTA                  
        sgd10 sgd15 sgd19   all sgd10 sgd15 sgd19   all
0 10SYN  83.4  80.4  74.7  77.7  30.6  17.0  14.5  17.2
  15SYN  83.1  85.1  76.3  79.9  44.2  23.9  19.9  24.1
  SYN    81.3  84.8  81.6  82.5  33.5  27.2  23.6  25.8
4 10SYN  87.0  84.1  82.1  83.5  38.0  27.7  21.0  24.8
  15SYN  85.3  85.5  83.0  84.1  32.2  27.4  20.4  23.7
  SYN    87.5  85.6  83.7  84.8  38.4  27.2  24.3  26.9

In [126]:
# domain_expand_table = domain_expand_table.drop(columns=['CDTA'])
domain_expand_table

RSA                    CDTA                  
        sgd10 sgd15 sgd19   all sgd10 sgd15 sgd19   all
0 10SYN  83.4  80.4  74.7  77.7  29.4  21.0  16.6  19.6
  15SYN  83.1  85.1  76.3  79.9  39.2  30.2  22.0  26.4
  SYN    81.3  84.8  81.6  82.5  32.0  34.9  26.5  29.0
4 10SYN  87.0  84.1  82.1  83.5  41.2  35.1  24.5  29.3
  15SYN  85.3  85.5  83.0  84.1  37.8  35.6  24.1  28.6
  SYN    87.5  85.6  83.7  84.8  42.4  34.6  27.0  31.0

In [123]:
domain_expand_table

RSA                    CDTA                  
         sgd10 sgd15 sgd19   all sgd10 sgd15 sgd19   all
0 SYN     81.3  84.8  81.6  82.5  32.0  34.9  26.5  29.0
  10TURE  92.3  68.1  60.9  68.0  70.0   7.8   3.8  15.5
  15TRUE  89.5  87.9  69.8  78.2  57.1  50.7   9.4  25.3
  19TRUE  82.5  87.4  87.0  86.4  62.8  57.6  45.9  50.9
4 SYN     87.5  85.6  83.7  84.8  42.4  34.6  27.0  31.0
  10TURE  92.3  85.5  85.1  86.4  78.4  28.0  25.3  34.5
  15TRUE  88.6  88.3  84.6  86.3  62.2  54.4  28.0  38.8
  19TRUE  86.0  86.4  87.0  86.7  45.2  50.5  40.0  42.9

In [24]:
print(domain_expand_table.to_latex())

\begin{tabular}{llrrrrrrrr}
\toprule
  &     & \multicolumn{4}{l}{RSA} & \multicolumn{4}{l}{CDTA} \\
  &     & sgd10 & sgd15 & sgd19 &   all & sgd10 & sgd15 & sgd19 &   all \\
\midrule
0 & 10SYN &  83.4 &  80.4 &  74.7 &  77.7 &  30.6 &  17.0 &  14.5 &  17.2 \\
  & 15SYN &  83.1 &  85.1 &  76.3 &  79.9 &  44.2 &  23.9 &  19.9 &  24.1 \\
  & SYN &  81.3 &  84.8 &  81.6 &  82.5 &  33.5 &  27.2 &  23.6 &  25.8 \\
4 & 10SYN &  87.0 &  84.1 &  82.1 &  83.5 &  38.0 &  27.7 &  21.0 &  24.8 \\
  & 15SYN &  85.3 &  85.5 &  83.0 &  84.1 &  32.2 &  27.4 &  20.4 &  23.7 \\
  & SYN &  87.5 &  85.6 &  83.7 &  84.8 &  38.4 &  27.2 &  24.3 &  26.9 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_50930/705736087.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(domain_expand_table.to_latex())


### CoQR

In [45]:
aug_types = [(2, 2.0), (2, 2.0), (2, 2.0), (2, 2.0), (4, 2.0), (5, 2.0), (5, 1.0), (5, 4.0), (11, 2.0), (1, 0.1)]
index_names = ['\singleConcat', 'CoQR-Zero', 'CoQR-ChatGPT', 'CoQR-SYN', '\singleConcatRel', '\singleSYN', '\singleSYN (1x)', '\singleSYN (4x)', '\singleSYNN', 'Real']
datasets = ['sgd/group0', 'multiwoz21']
coqr_test_table = pd.DataFrame(index=index_names, 
                               columns=pd.MultiIndex.from_product([datasets, ['JGA', 'RSA', 'CDTA']]))
                            #    columns=pd.MultiIndex.from_product([datasets, column_names, ['all']+[f'domain_{i}' for i in range(1,5)]]))
coqr_test_table

sgd/group0           multiwoz21          
                        JGA  RSA CDTA        JGA  RSA CDTA
\singleConcat           NaN  NaN  NaN        NaN  NaN  NaN
CoQR-Zero               NaN  NaN  NaN        NaN  NaN  NaN
CoQR-ChatGPT            NaN  NaN  NaN        NaN  NaN  NaN
CoQR-SYN                NaN  NaN  NaN        NaN  NaN  NaN
\singleConcatRel        NaN  NaN  NaN        NaN  NaN  NaN
\singleSYN              NaN  NaN  NaN        NaN  NaN  NaN
\singleSYN (1x)         NaN  NaN  NaN        NaN  NaN  NaN
\singleSYN (4x)         NaN  NaN  NaN        NaN  NaN  NaN
\singleSYNN             NaN  NaN  NaN        NaN  NaN  NaN
Real                    NaN  NaN  NaN        NaN  NaN  NaN

In [46]:
table = coqr_test_table
for i, (aug_type, aug_times) in enumerate(aug_types):
    index_name = index_names[i]
    for dataset in datasets:
        res_dir = f'output0615/{dataset}/qadst_f1_th>0.1/t5-large_aug{aug_type}_x{aug_times}_model0_context100'
        if index_name == 'CoQR-Zero':
            res_file = f'{res_dir}/test_multi_domain_coqr_canard_result_ori_cdta.md'
        elif index_name == 'CoQR-SYN':
            res_file = f'{res_dir}/test_multi_domain_coqr_result_ori_cdta.md'
        elif index_name == 'CoQR-ChatGPT':
            res_file = f'{res_dir}/test_multi_domain_coqr_chatgpt_result_ori_cdta.md'
        else:
            res_file = f'{res_dir}/test_multi_domain_result_ori_cdta.md'
        if not os.path.exists(res_file):
            print(res_file)
            continue
        df = pd.read_table(res_file, sep='|', index_col=1).dropna(axis=1, how='all').iloc[1:]
        df = df.apply(lambda x: x.str.strip())
        df = df.rename(columns=str.strip).rename(index=str.strip)
        table.loc[index_name, (dataset, 'RSA')] = df.loc['all', 'RSA']
        table.loc[index_name, (dataset, 'CDTA')] = df.loc['all', 'CDTA']
        table.loc[index_name, (dataset, 'JGA')] = df.loc['all', 'JGA']
        # table.loc[index_name, (dataset, 'SRSA')] = df.loc['domain_1', 'RSA']
        # res_file = res_file.replace('test_multi_domain', 'test_single_domain')
        # df = pd.read_table(res_file, sep='|', index_col=1).dropna(axis=1, how='all').iloc[1:]
        # df = df.apply(lambda x: x.str.strip())
        # df = df.rename(columns=str.strip).rename(index=str.strip)
        # table.loc[index_name, (dataset, 'SRSA')] = df.loc['all', 'RSA']
        # table.loc[index_name, (dataset, 'RSA', 'all')] = df.loc['all', 'RSA']
        # table.loc[index_name, (dataset, 'CDTA', 'all')] = df.loc['all', 'CDTA']
        # for dom_cnt in range(1, 5):
        #     table.loc[index_name, (dataset, 'RSA', f'domain_{dom_cnt}')] = df.loc[f'domain_{dom_cnt}', 'RSA']
        #     table.loc[index_name, (dataset, 'CDTA', f'domain_{dom_cnt}')] = df.loc[f'domain_{dom_cnt}', 'CDTA']

coqr_test_table = coqr_test_table.applymap(lambda x: round(pd.to_numeric(x, errors='coerce') * 100, 1))

output0615/sgd/group0/qadst_f1_th>0.1/t5-large_aug2_x2.0_model0_context100/test_multi_domain_coqr_canard_result_ori_cdta.md
output0615/multiwoz21/qadst_f1_th>0.1/t5-large_aug2_x2.0_model0_context100/test_multi_domain_coqr_canard_result_ori_cdta.md
output0615/sgd/group0/qadst_f1_th>0.1/t5-large_aug2_x2.0_model0_context100/test_multi_domain_coqr_chatgpt_result_ori_cdta.md
output0615/multiwoz21/qadst_f1_th>0.1/t5-large_aug2_x2.0_model0_context100/test_multi_domain_coqr_chatgpt_result_ori_cdta.md
output0615/sgd/group0/qadst_f1_th>0.1/t5-large_aug2_x2.0_model0_context100/test_multi_domain_coqr_result_ori_cdta.md
output0615/multiwoz21/qadst_f1_th>0.1/t5-large_aug2_x2.0_model0_context100/test_multi_domain_coqr_result_ori_cdta.md
output0615/sgd/group0/qadst_f1_th>0.1/t5-large_aug4_x2.0_model0_context100/test_multi_domain_result_ori_cdta.md
output0615/multiwoz21/qadst_f1_th>0.1/t5-large_aug4_x2.0_model0_context100/test_multi_domain_result_ori_cdta.md
output0615/sgd/group0/qadst_f1_th>0.1/t5-lar

In [47]:
coqr_test_table # model0

sgd/group0             multiwoz21            
                        JGA   RSA  CDTA        JGA   RSA  CDTA
\singleConcat          45.6  80.5   1.6       43.4  81.6  11.2
CoQR-Zero               NaN   NaN   NaN        NaN   NaN   NaN
CoQR-ChatGPT            NaN   NaN   NaN        NaN   NaN   NaN
CoQR-SYN                NaN   NaN   NaN        NaN   NaN   NaN
\singleConcatRel        NaN   NaN   NaN        NaN   NaN   NaN
\singleSYN             50.5  82.5  25.8       40.4  81.2  20.9
\singleSYN (1x)         NaN   NaN   NaN        NaN   NaN   NaN
\singleSYN (4x)         NaN   NaN   NaN        NaN   NaN   NaN
\singleSYNN             NaN   NaN   NaN        NaN   NaN   NaN
Real                   58.0  86.4  50.5        NaN   NaN   NaN

In [38]:
coqr_test_table # model4

sgd/group0       multiwoz21      
                        RSA  CDTA        RSA  CDTA
\singleConcat          82.5   1.8       83.2  11.1
CoQR-Zero              75.8  16.6       78.9  11.4
CoQR-ChatGPT           80.5  26.8       81.4  24.7
CoQR-SYN                NaN   NaN        NaN   NaN
\singleConcatRel       82.8   4.4       82.7  20.2
\singleSYN             84.4  21.1       81.5  27.6
\singleSYN (1x)        85.0  24.0       81.7  28.4
\singleSYN (4x)        83.8  26.5       81.2  29.8
\singleSYNN            85.5  27.4       82.0  30.1
Real                   86.7  39.6       83.4  31.4

In [30]:
coqr_test_table # model7

sgd/group0       multiwoz21      
                        RSA  CDTA        RSA  CDTA
\singleConcat          84.0   1.7       84.1   8.3
CoQR-Zero              76.5  17.9       79.6   9.4
CoQR-ChatGPT           81.2  27.3       82.3  24.8
CoQR-SYN               85.6  17.8       83.8  22.6
\singleConcatRel        NaN   NaN        NaN   NaN
\singleSYN             86.0  19.7       83.2  27.7
\singleSYN (1x)         NaN   NaN        NaN   NaN
\singleSYN (4x)         NaN   NaN        NaN   NaN
\singleSYNN             NaN   NaN        NaN   NaN
Real                    NaN   NaN        NaN   NaN

In [34]:
coqr_test_table # model5

sgd/group0       multiwoz21      
                        RSA  CDTA        RSA  CDTA
\singleConcat          80.9   0.5       81.8   7.3
CoQR-Zero              73.3  16.0       77.5   8.1
CoQR-ChatGPT           78.4  25.7       80.0  23.0
CoQR-SYN               82.7  16.9       81.5  21.6
\singleConcatRel        NaN   NaN        NaN   NaN
\singleSYN             82.7  21.8       80.7  29.1
\singleSYN (1x)         NaN   NaN        NaN   NaN
\singleSYN (4x)         NaN   NaN        NaN   NaN
\singleSYNN             NaN   NaN        NaN   NaN
Real                    NaN   NaN        NaN   NaN

In [40]:
# coqr_test_table = coqr_test_table.drop(columns=[('multiwoz21', 'RSA', 'domain_4'),('multiwoz21', 'CDTA', 'domain_4'),
#                                               ('multiwoz21', 'CDTA', 'domain_1'), 
#                                             #   ('multiwoz21', 'CDTA', 'domain_3'), ('multiwoz21', 'CDTA', 'domain_4'),
#                                               ('sgd/group0', 'CDTA', 'domain_1'), 
#                                             #   ('sgd/group0', 'CDTA', 'domain_3'), ('sgd/group0', 'CDTA', 'domain_4'),
#                                               ])
coqr_test_table # 0620 ori cdta

sgd/group0       multiwoz21      
                        RSA  CDTA        RSA  CDTA
\singleConcat          82.5   1.8       83.2  11.1
CoQR-Zero              75.8  16.6       78.9  11.4
CoQR-ChatGPT           80.5  26.8       81.4  24.7
CoQR-SYN               84.1  17.2       83.1  24.0
\singleConcatRel       82.8   4.4       82.7  20.2
\singleSYN             84.8  26.9       82.2  30.8
\singleSYN (1x)        85.0  24.0       81.7  28.4
\singleSYN (4x)        83.8  26.5       81.2  29.8
\singleSYNN            85.5  27.4       82.0  30.1
Real                   86.7  39.6       83.4  31.4

In [165]:
coqr_test_table = coqr_test_table.drop(columns=[('multiwoz21', 'RSA', 'domain_4'),('multiwoz21', 'CDTA', 'domain_4'),
                                              ('multiwoz21', 'CDTA', 'domain_1'), 
                                            #   ('multiwoz21', 'CDTA', 'domain_3'), ('multiwoz21', 'CDTA', 'domain_4'),
                                              ('sgd/group0', 'CDTA', 'domain_1'), 
                                            #   ('sgd/group0', 'CDTA', 'domain_3'), ('sgd/group0', 'CDTA', 'domain_4'),
                                              ])
# for i, (aug_type, aug_times) in enumerate(aug_types):
#     index_name = index_names[i]
#     main_exp_table = main_exp_table.drop(index=('MinTL', index_name, 4))
coqr_test_table

sgd/group0                                            \
                        RSA                                      CDTA   
                        all domain_1 domain_2 domain_3 domain_4   all   
\singleConcat          82.5     93.9     76.9     74.7     50.9   8.8   
CoQR-Zero              75.8     84.3     71.5     70.7     47.3  20.0   
CoQR-SYN               84.1     93.6     79.6     76.5     58.0  22.9   
\singleSYN             84.8     92.9     81.3     77.0     66.7  31.0   
\singleConcatRel        NaN      NaN      NaN      NaN      NaN   NaN   

                                            multiwoz21                    \
                                                   RSA                     
                 domain_2 domain_3 domain_4        all domain_1 domain_2   
\singleConcat         7.8     16.3      1.5       83.2     85.3     83.6   
CoQR-Zero            19.2     23.5     23.1       78.9     81.9     78.4   
CoQR-SYN             23.5     17.0     33.8       83.1     85.4     82.5   
\singleSYN           32.5     20.8     36.9       82.2     84.8     81.8   
\singleConcatRel      NaN      NaN      NaN        NaN      NaN      NaN   

                                                   
                           CDTA                    
                 domain_3   all domain_2 domain_3  
\singleConcat        68.4  37.3     49.9     10.1  
CoQR-Zero            64.9  33.3     43.0     12.5  
CoQR-SYN             73.2  44.0     51.2     28.5  
\singleSYN           70.4  45.5     56.8     21.2  
\singleConcatRel      NaN   NaN      NaN      NaN

In [135]:
coqr_test_table

sgd/group0       multiwoz21      
                        RSA  CDTA        RSA  CDTA
\singleConcat          82.5   8.8       83.2  37.3
\singleConcatRel        NaN   NaN        NaN   NaN
\singleSYN             84.8  31.0       82.2  45.5
\singleSYNN            85.5  31.9       82.0  46.1
Real                   86.7  42.9       83.4  48.0
CoQR-C                 75.8  20.0       78.9  33.3
CoQR-S                 84.1  22.9       83.1  44.0
\singleSYN (1x)        85.0  27.9       81.7  43.5
\singleSYN (4x)        83.8  30.7       81.2  43.9

In [35]:
print(coqr_test_table.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{sgd/group0} & \multicolumn{2}{l}{multiwoz21} \\
{} &        RSA &  CDTA &        RSA &  CDTA \\
\midrule
\textbackslash singleConcat    &       80.9 &   0.5 &       81.8 &   7.3 \\
CoQR-Zero        &       73.3 &  16.0 &       77.5 &   8.1 \\
CoQR-ChatGPT     &       78.4 &  25.7 &       80.0 &  23.0 \\
CoQR-SYN         &       82.7 &  16.9 &       81.5 &  21.6 \\
\textbackslash singleConcatRel &        NaN &   NaN &        NaN &   NaN \\
\textbackslash singleSYN       &       82.7 &  21.8 &       80.7 &  29.1 \\
\textbackslash singleSYN (1x)  &        NaN &   NaN &        NaN &   NaN \\
\textbackslash singleSYN (4x)  &        NaN &   NaN &        NaN &   NaN \\
\textbackslash singleSYNN      &        NaN &   NaN &        NaN &   NaN \\
Real             &        NaN &   NaN &        NaN &   NaN \\
\bottomrule
\end{tabular}



/tmp/ipykernel_33250/3055608670.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(coqr_test_table.to_latex())


In [113]:
print(coqr_test_table.loc['turn'].to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{multiwoz21} & \multicolumn{5}{l}{sgd/group0} \\
{} &  Multi JGA & Multi TA & Multi CDTA & Cross JGA & Single JGA &  Multi JGA & Multi TA & Multi CDTA & Cross JGA & Single JGA \\
\midrule
Single+Concat (2x) &       39.5 &     74.1 &        5.6 &      28.9 &       63.6 &       36.3 &     78.3 &        1.3 &      35.2 &       83.3 \\
w/ CANARD          &       34.5 &     70.3 &        6.2 &      26.6 &       59.2 &       30.0 &     71.0 &       13.7 &      30.4 &       64.6 \\
w/ CANARD+MIX      &       39.4 &     73.7 &       16.6 &      30.0 &       63.4 &       39.5 &     78.7 &       18.1 &      39.5 &       82.2 \\
w/ ChatGPT         &       36.3 &     70.6 &       17.0 &      27.4 &       61.1 &       32.3 &     72.7 &       22.7 &      32.1 &       66.2 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_88698/2481759033.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(coqr_test_table.loc['turn'].to_latex())


### Ablation

In [80]:
qadst_dirs = ['qadst_f1_th>0', 'qadst_f1_th>0.01', 'qadst_f1_th>0.1', 'qadst_f1_th>0.3', 'qadst_f1_th>0.5', 'qadst_f1_th>0.8', 'qadst_f1_th>0.1_true_domain_comb', 'qadst_true_slot_pairs']
aug_types = [(5, 2.0)]*len(qadst_dirs)
index_names = ['F1>0', 'F1>0.01', 'F1>0.1', 'F1>0.3', 'F1>0.5', 'F1>0.8', 'F1>0.1&TRUE_DOM', 'TRUE_SLOT']
datasets = ['sgd/group0', 'multiwoz21']
ablation_table = pd.DataFrame(index=index_names, 
                               columns=pd.MultiIndex.from_product([datasets, ['RSA', 'CDTA', 'Ratio', 'Precision', 'Recall']]))
                            #    columns=pd.MultiIndex.from_product([datasets, column_names, ['all']+[f'domain_{i}' for i in range(1,5)]]))
ablation_table

sgd/group0                             multiwoz21             \
                       RSA CDTA Ratio Precision Recall        RSA CDTA Ratio   
F1>0                   NaN  NaN   NaN       NaN    NaN        NaN  NaN   NaN   
F1>0.01                NaN  NaN   NaN       NaN    NaN        NaN  NaN   NaN   
F1>0.1                 NaN  NaN   NaN       NaN    NaN        NaN  NaN   NaN   
F1>0.3                 NaN  NaN   NaN       NaN    NaN        NaN  NaN   NaN   
F1>0.5                 NaN  NaN   NaN       NaN    NaN        NaN  NaN   NaN   
F1>0.8                 NaN  NaN   NaN       NaN    NaN        NaN  NaN   NaN   
F1>0.1&TRUE_DOM        NaN  NaN   NaN       NaN    NaN        NaN  NaN   NaN   
TRUE_SLOT              NaN  NaN   NaN       NaN    NaN        NaN  NaN   NaN   

                                  
                Precision Recall  
F1>0                  NaN    NaN  
F1>0.01               NaN    NaN  
F1>0.1                NaN    NaN  
F1>0.3                NaN    NaN  
F1>0.5                NaN    NaN  
F1>0.8                NaN    NaN  
F1>0.1&TRUE_DOM       NaN    NaN  
TRUE_SLOT             NaN    NaN

In [81]:
table = ablation_table
for i, (aug_type, aug_times) in enumerate(aug_types):
    qadst_dir = qadst_dirs[i]
    index_name = index_names[i]
    for dataset in datasets:
        res_dir = f'output0615/{dataset}/{qadst_dir}/t5-large_aug{aug_type}_x{aug_times}_model4_context100'
        res_file = f'{res_dir}/test_multi_domain_result_ori_cdta.md'
        if not os.path.exists(res_file):
            continue
        df = pd.read_table(res_file, sep='|', index_col=1).dropna(axis=1, how='all').iloc[1:]
        df = df.apply(lambda x: x.str.strip())
        df = df.rename(columns=str.strip).rename(index=str.strip)
        table.loc[index_name, (dataset, 'RSA')] = df.loc['all', 'RSA']
        table.loc[index_name, (dataset, 'CDTA')] = df.loc['all', 'CDTA']
        # table.loc[index_name, (dataset, 'RSA', 'all')] = df.loc['all', 'RSA']
        # table.loc[index_name, (dataset, 'CDTA', 'all')] = df.loc['all', 'CDTA']
        # for dom_cnt in range(1, 5):
        #     table.loc[index_name, (dataset, 'RSA', f'domain_{dom_cnt}')] = df.loc[f'domain_{dom_cnt}', 'RSA']
        #     table.loc[index_name, (dataset, 'CDTA', f'domain_{dom_cnt}')] = df.loc[f'domain_{dom_cnt}', 'CDTA']



In [82]:
ablation_table

sgd/group0                                       \
                                RSA                 CDTA Ratio Precision   
F1>0             0.8455318300347572  0.26227738801942796   NaN       NaN   
F1>0.01          0.8430816095835237  0.24662709120345386   NaN       NaN   
F1>0.1           0.8483298514948231  0.26875337290879653   NaN       NaN   
F1>0.3           0.8515074526539687  0.23961144090663788   NaN       NaN   
F1>0.5           0.8508244686320376  0.26227738801942796   NaN       NaN   
F1>0.8           0.8392410054259714  0.18078791149487317   NaN       NaN   
F1>0.1&TRUE_DOM  0.8449357245312612  0.24986508364813814   NaN       NaN   
TRUE_SLOT        0.8642971086118983   0.3059902860226659   NaN       NaN   

                                multiwoz21                             \
                Recall                 RSA                 CDTA Ratio   
F1>0               NaN  0.8189786188385456  0.29117876658860264   NaN   
F1>0.01            NaN  0.8250100191216666   0.3091334894613583   NaN   
F1>0.1             NaN   0.822339879389787   0.3075722092115535   NaN   
F1>0.3             NaN   0.818364395259508  0.30601092896174864   NaN   
F1>0.5             NaN  0.8147157497006207   0.2693208430913349   NaN   
F1>0.8             NaN  0.8165034655316281   0.2154566744730679   NaN   
F1>0.1&TRUE_DOM    NaN  0.8185428005688219  0.29430132708821233   NaN   
TRUE_SLOT          NaN  0.8268752468589738   0.3473848555815769   NaN   

                                  
                Precision Recall  
F1>0                  NaN    NaN  
F1>0.01               NaN    NaN  
F1>0.1                NaN    NaN  
F1>0.3                NaN    NaN  
F1>0.5                NaN    NaN  
F1>0.8                NaN    NaN  
F1>0.1&TRUE_DOM       NaN    NaN  
TRUE_SLOT             NaN    NaN

In [83]:
table = ablation_table
for dataset in datasets:
    print(dataset)
    golden_slot_pairs = json.load(open(f'data/{dataset}/multi_domain_slot_pairs.json'))
    print(len(golden_slot_pairs))
    # total_slot_pairs = 143 if dataset == 'multiwoz21' else 889
    total_slot_pairs = 690 if dataset == 'multiwoz21' else 9972
    for index_name, qadst_dir in zip(index_names, qadst_dirs):
        pred_slot_pairs = json.load(open(f'data/{dataset}/{qadst_dir}/aug5_x2.0/coqr_slot_pairs.json'))
        precision, recall, f1 = eval_slot_pairs_prediction(set(golden_slot_pairs.keys()), set(pred_slot_pairs.keys()))
        # print(f'{qadst_dir}, precision={precision:.2f}, recall={recall:.2f}, f1={f1:.2f}, size={len(pred_slot_pairs)}, filtered_ratio={len(pred_slot_pairs)/total_slot_pairs:.2f}')
        table.loc[index_name, (dataset, 'Ratio')] = len(pred_slot_pairs)/total_slot_pairs
        table.loc[index_name, (dataset, 'Precision')] = precision
        table.loc[index_name, (dataset, 'Recall')] = recall

ablation_table = ablation_table.applymap(lambda x: round(pd.to_numeric(x, errors='coerce') * 100, 1))

sgd/group0
87
multiwoz21
34


In [84]:
ablation_table

sgd/group0                              multiwoz21        \
                       RSA  CDTA Ratio Precision Recall        RSA  CDTA   
F1>0                  84.6  26.2   4.7      10.9   58.6       81.9  29.1   
F1>0.01               84.3  24.7   4.7      10.9   58.6       82.5  30.9   
F1>0.1                84.8  26.9   4.5      10.8   55.2       82.2  30.8   
F1>0.3                85.2  24.0   4.1      12.0   56.3       81.8  30.6   
F1>0.5                85.1  26.2   3.8      11.4   49.4       81.5  26.9   
F1>0.8                83.9  18.1   2.8      14.9   47.1       81.7  21.5   
F1>0.1&TRUE_DOM       84.5  25.0   1.3      48.1   73.6       81.9  29.4   
TRUE_SLOT             86.4  30.6   0.7     100.0   75.9       82.7  34.7   

                                        
                Ratio Precision Recall  
F1>0             14.9      29.1   88.2  
F1>0.01          14.1      33.0   94.1  
F1>0.1           11.2      35.1   79.4  
F1>0.3            8.7      41.7   73.5  
F1>0.5            7.1      46.9   67.6  
F1>0.8            2.5      64.7   32.4  
F1>0.1&TRUE_DOM   9.1      41.3   76.5  
TRUE_SLOT         4.8     100.0   97.1

In [78]:
ablation_table.drop(index='F1>0.1&TRUE_DOM').drop(index='F1>0.01')

sgd/group0                              multiwoz21              \
                 RSA  CDTA Ratio Precision Recall        RSA  CDTA Ratio   
F1>0            84.6  26.2   4.7      10.9   58.6       81.9  29.1  14.9   
F1>0.1          84.8  26.9   4.5      10.8   55.2       82.2  30.8  11.2   
F1>0.3          85.2  24.0   4.1      12.0   56.3       81.8  30.6   8.7   
F1>0.5          85.1  26.2   3.8      11.4   49.4       81.5  26.9   7.1   
F1>0.8          83.9  18.1   2.8      14.9   47.1       81.7  21.5   2.5   
TRUE_SLOT       86.4  30.6   0.7     100.0   75.9       82.7  34.7   4.8   

                            
          Precision Recall  
F1>0           29.1   88.2  
F1>0.1         35.1   79.4  
F1>0.3         41.7   73.5  
F1>0.5         46.9   67.6  
F1>0.8         64.7   32.4  
TRUE_SLOT     100.0   97.1

In [79]:
print(ablation_table.drop(index='F1>0.1&TRUE_DOM').drop(index='F1>0.01').to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{sgd/group0} & \multicolumn{5}{l}{multiwoz21} \\
{} &        RSA &  CDTA & Ratio & Precision & Recall &        RSA &  CDTA & Ratio & Precision & Recall \\
\midrule
F1>0      &       84.6 &  26.2 &   4.7 &      10.9 &   58.6 &       81.9 &  29.1 &  14.9 &      29.1 &   88.2 \\
F1>0.1    &       84.8 &  26.9 &   4.5 &      10.8 &   55.2 &       82.2 &  30.8 &  11.2 &      35.1 &   79.4 \\
F1>0.3    &       85.2 &  24.0 &   4.1 &      12.0 &   56.3 &       81.8 &  30.6 &   8.7 &      41.7 &   73.5 \\
F1>0.5    &       85.1 &  26.2 &   3.8 &      11.4 &   49.4 &       81.5 &  26.9 &   7.1 &      46.9 &   67.6 \\
F1>0.8    &       83.9 &  18.1 &   2.8 &      14.9 &   47.1 &       81.7 &  21.5 &   2.5 &      64.7 &   32.4 \\
TRUE\_SLOT &       86.4 &  30.6 &   0.7 &     100.0 &   75.9 &       82.7 &  34.7 &   4.8 &     100.0 &   97.1 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_103566/327982617.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(ablation_table.drop(index='F1>0.1&TRUE_DOM').drop(index='F1>0.01').to_latex())
